In [11]:
import json
import pandas as pd

from glob import glob
from concurrent.futures import ThreadPoolExecutor

data_root = 'data/gamepress/pokemons'

def process_pokemon_file(pokemon_file):
	try:
		with open(pokemon_file) as f:
			pokemon_data = json.load(f)
			weaknesses = pokemon_data.get('weaknesses', [])
			# resistances = pokemon_data.get('resistances', [])
			# weakness_str = ','.join([_['type'] for _ in weaknesses])
			# not_resistance_str = '&'.join([f'!{_["type"]}' for _ in resistances])
			# search_str = f'{not_resistance_str}&{weakness_str}'

			skills = []
			for i in range(1, 4):
				for _ in [_['type'] for _ in weaknesses]:
					skills.append(f'@{i}{_}')

			search_str = ','.join(skills[:len(weaknesses)]) + '&' + ','.join(skills[len(weaknesses):])

			return {
				'name': pokemon_data.get('name'),
				'search': search_str,
				'type': ','.join(pokemon_data.get('types', [])),
				# 'fast_moves': ','.join([_['name'] for _ in pokemon_data.get('fast_moves', [])]),
				# 'charge_moves': ','.join([_['name'] for _ in pokemon_data.get('charge_moves', [])]),
				# 'weaknesses': weakness_str,
				# 'resistances': ','.join([_['type'] for _ in resistances]),
				# 'attack': pokemon_data.get('attack'),
				# 'defense': pokemon_data.get('defense'),
				# 'stamina': pokemon_data.get('stamina'),
				'max_cp': pokemon_data.get('max_cp')
			}
	except Exception as e:
		print(f'Error processing {pokemon_file}. {e}')

with ThreadPoolExecutor() as executor:
	pokemon_files = glob(f'{data_root}/*.json')
	print(f'No. of pokemon files: {len(pokemon_files)}')
	pokemon_datas = list(executor.map(process_pokemon_file, pokemon_files))

pokemons_df = pd.DataFrame(pokemon_datas)

pokemons_df.to_json(f'data/json/individual_pokemon.json', orient='records')
pokemons_df

No. of pokemon files: 1616


,name,search,type,max_cp
0,5th Anniversary Pikachu,"@1Ground&@2Ground,@3Ground",Electric,1060
1,Abomasnow,"@1Fire,@1Bug,@1Fighting,@1Flying,@1Poison,@1Ro...","Grass,Ice",2670
2,Abra,"@1Bug,@1Dark,@1Ghost&@2Bug,@2Dark,@2Ghost,@3Bu...",Psychic,1517
3,Absol,"@1Bug,@1Fairy,@1Fighting&@2Bug,@2Fairy,@2Fight...",Dark,2856
4,Accelgor,"@1Fire,@1Flying,@1Rock&@2Fire,@2Flying,@2Rock,...",Bug,2760
...,...,...,...,...
1611,Zubat,"@1Electric,@1Ice,@1Psychic,@1Rock&@2Electric,@...","Flying,Poison",754
1612,Zweilous,"@1Fairy,@1Dragon,@1Bug,@1Fighting,@1Ice&@2Fair...","Dark,Dragon",2079
1613,Zygarde (10% Forme),"@1Ice,@1Dragon,@1Fairy&@2Ice,@2Dragon,@2Fairy,...","Dragon,Ground",2685
1614,Zygarde (50% Forme),"@1Ice,@1Dragon,@1Fairy&@2Ice,@2Dragon,@2Fairy,...","Dragon,Ground",3855
